# Proyecto: Análisis de Comportamiento de Usuarios

Este notebook realiza los siguientes pasos:

1. Carga y exploración del dataset
2. División en conjuntos de entrenamiento, validación y prueba
3. Entrenamiento y ajuste de hiperparámetros de un modelo (Random Forest)
4. Evaluación del modelo en el conjunto de validación y prueba
5. Prueba de cordura del modelo

El dataset (`users_behavior.csv`) contiene la siguiente información para cada usuario:
- `calls`: número de llamadas
- `minutes`: duración total de llamadas en minutos
- `messages`: número de mensajes de texto
- `mb_used`: tráfico de Internet utilizado en MB
- `is_ultra`: plan del mes actual (Ultra - 1, Smart - 0)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Cargar el dataset
data = pd.read_csv("/datasets/users_behavior.csv")

# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
print(data.head())

# Mostrar estadísticas descriptivas
print("Estadísticas descriptivas:")
print(data.describe())

# Información general del dataset
print("Información del dataset:")
print(data.info())

## División del Conjunto de Datos

Separamos las características (`calls`, `minutes`, `messages`, `mb_used`) de la variable objetivo (`is_ultra`) y dividimos el dataset en entrenamiento (60%), validación (20%) y prueba (20%).

In [ ]:
# Separar características y variable objetivo
X = data[['calls', 'minutes', 'messages', 'mb_used']]
y = data['is_ultra']

# Primera división: entrenamiento (60%) y conjunto temporal (40%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

# Segunda división: el conjunto temporal se divide en validación (20%) y prueba (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

## Entrenamiento y Ajuste de Hiperparámetros

Utilizaremos un **Random Forest** y `GridSearchCV` para encontrar la mejor combinación de hiperparámetros.

In [ ]:
# Definir el clasificador y la grilla de hiperparámetros
rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Configurar GridSearchCV
grid_rf = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
grid_rf.fit(X_train, y_train)

# Mejor modelo encontrado
best_rf = grid_rf.best_estimator_
print("Mejores hiperparámetros:", grid_rf.best_params_)

# Evaluar en el conjunto de validación
y_val_pred = best_rf.predict(X_val)
print("Accuracy en validación:", accuracy_score(y_val, y_val_pred))
print("Reporte de clasificación en validación:")
print(classification_report(y_val, y_val_pred))

## Evaluación Final en el Conjunto de Prueba

Una vez seleccionado el mejor modelo basado en el conjunto de validación, se evalúa su desempeño final en el conjunto de prueba.

In [ ]:
# Evaluación final en el conjunto de prueba
y_test_pred = best_rf.predict(X_test)
print("Accuracy en prueba:", accuracy_score(y_test, y_test_pred))
print("Reporte de clasificación en prueba:")
print(classification_report(y_test, y_test_pred))

## Prueba de Cordura

Para verificar que el modelo tiene un comportamiento razonable, examinamos la distribución de las predicciones en el conjunto de prueba.

In [ ]:
# Verificar la distribución de predicciones
unique, counts = np.unique(y_test_pred, return_counts=True)
plt.figure()
plt.bar(unique, counts)
plt.xlabel("Clase predicha")
plt.ylabel("Frecuencia")
plt.title("Distribución de predicciones en el conjunto de prueba")
plt.show()

## Conclusiones

En este notebook se ha:

- Explorado y analizado el dataset
- Dividido los datos en entrenamiento, validación y prueba
- Ajustado un modelo de Random Forest mediante GridSearchCV
- Evaluado el modelo en conjuntos de validación y prueba
    - Se han analizado métricas como la precisión, recall y F1-score
- Realizado una prueba de cordura para verificar la distribución de las predicciones

Estos pasos permiten obtener un modelo robusto y validar que se comporta de manera coherente con los datos y la problemática planteada.